### This genetic algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

In [2]:
from qutip import *
from scipy import arcsin, sqrt, pi, sin, cos
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy
from math import log2
from scipy.optimize import minimize
from IPython.core.display import clear_output

Using matplotlib backend: Qt5Agg


In [22]:
INPUT_STATES = 6 # at most 16

In [23]:
def h(x):  #binary entropy
    return -x*log2(x) - (1-x)*log2(1-x)

def calc_XN(p):
    def calc_XN_tmp(q):
        tmp = h(q * (1-p)) - h(0.5*(1+sqrt(1-4*p*(1-p)*q**2)))
        return 1/tmp

    res = minimize(calc_XN_tmp, np.array([0.5]), method='nelder-mead', 
                       options={'xtol': 1e-8, 'disp': False});
    return 1/res.fun;


def A_p2(rho): # this is the amplitude dampening channel on 2 qubits (2 uses). rho is a 2-qubit state.
    #------Moved to main to optimize code
    #KA = tensor(K1,K1)
    #KA_dag = tensor(K1,K1).dag()
    #KB = tensor(K1,K2)
    #KB_dag = tensor(K1,K2).dag()
    #KC = tensor(K2,K1)
    #KC_dag = tensor(K2,K1).dag()
    #KD = tensor(K2,K2)
    #KD_dag = tensor(K2,K2).dag()
    #--------

    return  KA * rho * KA_dag + KB * rho * KB_dag + KC * rho * KC_dag + KD * rho * KD_dag 


def Func(concurrences, angles, ps):   #This is the estimate of X(N \tensor N)
    
    #rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()   #moved to main
    #----------make the states with the given concurrences
    XNN = []
    for i in range(POP_SIZE):
        first_term = 0
        second_term = 0
        for j in range(INPUT_STATES):
            theta = arcsin(concurrences[i,j])
            measure = cos(theta/2) * basis(2,0) + sin(theta/2) * basis(2,1)
            ket = (tensor(measure.dag(), identity(2), identity(2)) * ghz_state()).unit()
            ket_rotated = tensor(ry(angles[i,j,0] * 2 * pi), ry(angles[i,j,1] * 2 * pi)) * ket 
            first_term = first_term + ps[i,j] * A_p2(ket2dm(ket_rotated))
            second_term = second_term + ps[i,j] * entropy_vn(A_p2(ket2dm(ket_rotated)), base=2)
        first_term = entropy_vn(first_term, base=2)
        XNN.append(first_term - second_term)
    
    #XNN = np.array([ entropy_vn(sum(px * A_p2(ket2dm(ket_rotated)) for px,Ux in zip(ps[i],Us[i])), base=2) 
    #                - sum(px * entropy_vn(A_p2(Ux * rho_init * Ux.dag()), base=2) for px,Ux in zip(ps[i],Us[i])) for i in range(POP_SIZE)])
    
    #XNN = np.array([ entropy_vn(sum(px * A_p2(Ux * rho_init * Ux.dag()) for px,Ux in zip(ps[i],Us[i])), base=2) 
    #                - sum(px * entropy_vn(A_p2(Ux * rho_init * Ux.dag()), base=2) for px,Ux in zip(ps[i],Us[i])) for i in range(POP_SIZE)])

    return np.array(XNN)
    
# find non-zero fitness for selection.
def get_fitness(pred): return pred + 1e-3 - np.min(pred)


def translateDNA(pop):   #pop is a list of 2 np arrays. One nparray for the population of X and Y, and one nparray for the population of ps
    concurrences = pop[0].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    angles = pop[1].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    prob = pop[2].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    ps = np.array([prob[i] / sum(prob[i]) for i in range(POP_SIZE)] )    
    return [concurrences,angles,ps] 

# nature selection wrt pop's fitness.
def select(pop, fitness):    
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    A = pop[0][idx]
    B = pop[1][idx]
    C = pop[2][idx]
    A[0] = pop[0][np.argmax(fitness)]  #makes sure the individual with highest fitness is chosen
    B[0] = pop[1][np.argmax(fitness)]
    C[0] = pop[2][np.argmax(fitness)]
    return [A, B ,C]

# mating process (genes crossover).
def crossover_and_mutate(individual_conc, individual_rot, individual_p, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points_conc = np.random.randint(0, 2, size=INPUT_STATES*DNA_SIZE).astype(bool).reshape((INPUT_STATES,DNA_SIZE))# choose crossover points
        cross_points_rot = np.random.randint(0, 2, size=INPUT_STATES*2*DNA_SIZE).astype(bool).reshape((INPUT_STATES,2,DNA_SIZE))# choose crossover points
        cross_points_p = np.random.randint(0, 2, size=DNA_SIZE*INPUT_STATES).astype(bool).reshape((INPUT_STATES,DNA_SIZE))# choose crossover points
        individual_conc[cross_points_conc] = pop[0][i_][cross_points_conc]
        individual_rot[cross_points_rot] = pop[1][i_][cross_points_rot]
        individual_p[cross_points_p] = pop[2][i_][cross_points_p]
        
    #mutate
    if mutate and np.random.rand() < MUTATION_FRACTION:
        x_conc = np.random.choice([0, 1], size=INPUT_STATES*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,DNA_SIZE))
        x_rot = np.random.choice([0, 1], size=INPUT_STATES*2*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,2,DNA_SIZE))
        x_p = np.random.choice([0, 1], size=INPUT_STATES*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,DNA_SIZE))
                                                      

        individual_conc[x_conc] = np.abs(individual_conc[x_conc] - 1)  #flip the bits
        individual_rot[x_rot] = np.abs(individual_rot[x_rot] - 1)  #flip the bits
        individual_p[x_p] = np.abs(individual_p[x_p] - 1)  #flip the bits
        
    return [individual_conc, individual_rot, individual_p]


In [ ]:
p = 0.001                 #probability p in the amp. damp. channel
DNA_SIZE = 10          # DNA length   # size of each number in the lists X and Y
POP_SIZE = 10           # population size
CROSS_RATE = 0.005        # mating probability (DNA crossover)
MUTATION_RATE = 0.01    # mutation probability
MUTATION_FRACTION = 1   #percentage of children to mutate
N_GENERATIONS = 5000

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()

#rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()
#-------------------------

pop_conc = []
pop_rot = []
pop_p = []
for i in range(POP_SIZE):
    pop_conc.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
    pop_rot.append(np.random.randint(2, size=([INPUT_STATES, 2, DNA_SIZE])))
    pop_p.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
pop = [np.array(pop_conc), np.array(pop_rot), np.array(pop_p)]

#-----------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.4,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)
plt.plot([i for i in range(N_GENERATIONS)], 
         [2*XN for j in range(N_GENERATIONS)])
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    #-=----------variable mutation rate
    #if _ > 700: MUTATION_RATE = 0.001
    #---------------------
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1], translated[2])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    clear_output(wait=True)
    print("probabilities ", translated[2][np.argmax(F_values)]) 
    print("angles (2 pi) ", translated[1][np.argmax(F_values)]) 
    print("concurrences ", translated[0][np.argmax(F_values)])
    
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    pop_copy_2 = pop.copy()
    for parent_conc, parent_rot, parent_p in zip(pop[0], pop[1], pop[2]):
        child_conc, child_rot, child_p = crossover_and_mutate(parent_conc, parent_rot, parent_p, pop_copy, len(holevo)>1 and holevo[-1]==holevo[-2]) 
        #parent_conc[:] = child_conc       # parent is replaced by its child
        #parent_rot[:] = child_rot
        #parent_p[:] = child_p        # parent is replaced by its child
        pop_copy_2[0] = np.concatenate((pop_copy_2[0],child_conc[np.newaxis, ...]),axis=0)
        pop_copy_2[1] = np.concatenate((pop_copy_2[1],child_rot[np.newaxis, ...]),axis=0)
        pop_copy_2[2] = np.concatenate((pop_copy_2[2],child_p[np.newaxis, ...]),axis=0)
        #parent_XY[:] = child_XY       # parent is replaced by its child
        #parent_p[:] = child_p        # parent is replaced by its child
    pop[0] = pop_copy_2[0]
    pop[1] = pop_copy_2[1]
    pop[2] = pop_copy_2[2]


plt.ioff()

probabilities  [ 0.11837924  0.25582627  0.13506356  0.13532839  0.20550847  0.14989407]
angles (2 pi)  [[ 0.71358749  0.8797654 ]
 [ 0.          0.3773216 ]
 [ 0.485826    0.21798631]
 [ 0.49951124  0.44086022]
 [ 0.2971652   0.81231672]
 [ 0.89833822  0.87194526]]
concurrences  [ 0.02150538  0.          0.04887586  0.01857283  0.          0.25219941]


KeyboardInterrupt: 

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "C:\Users\